<a href="https://colab.research.google.com/github/Anu589/Amazon-Job-Review-Sentiment-Analysis/blob/main/Copy_of_Amazon_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# ==============================================================================
# 1. Project Setup and Imports
# ==============================================================================
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from transformers import T5ForConditionalGeneration, T5Tokenizer
import os
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import warnings
import kaleido

warnings.filterwarnings('ignore')

def save_chart(fig, filename):
    """
    Saves a figure to the 'charts' directory.
    - Plotly figures are saved as interactive HTML files.
    - Matplotlib figures are saved as static PNG images.
    """
    if not os.path.exists('charts'):
        os.makedirs('charts')
    if isinstance(fig, go.Figure):
        fig.write_html(f"charts/{filename}.html")
        print(f"Saved chart to charts/{filename}.html")
    else:
        fig.savefig(f"charts/{filename}.png")
        print(f"Saved chart to charts/{filename}.png")

# ==============================================================================
# 2. LLM Model and Vectorized Sentiment Analysis
# ==============================================================================
def load_llama_model():
    """
    Loads a Hugging Face T5 model and tokenizer for sentiment analysis.
    """
    model_name = "google/flan-t5-small"
    try:
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        tokenizer = T5Tokenizer.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        print(f"Error loading LLM: {e}")
        return None, None

model, tokenizer = load_llama_model()

def get_llama_sentiment(text_list, batch_size=32):
    """
    Analyzes a list of texts for sentiment using the T5 model with batching for efficiency.
    Returns a list of sentiment scores (-1, 0, 1).
    """
    if model is None or tokenizer is None:
        print("LLM not loaded. Skipping sentiment analysis.")
        return [0] * len(text_list)

    sentiments = []
    # This tqdm bar will now only show 32 batches (1000/32)
    for i in tqdm(range(0, len(text_list), batch_size), desc="Getting Llama Sentiments (Batched)"):
        batch = text_list[i:i + batch_size]

        prompts = [
            f"What is the sentiment of this review? ' {text} '. Answer with a single word: Positive, Negative, or Neutral."
            if isinstance(text, str) and text.strip() not in ["na", "nan", "none", "#name?", "no text provided"]
            else None for text in batch
        ]

        valid_prompts = [p for p in prompts if p is not None]
        if not valid_prompts:
            sentiments.extend([0] * len(batch))
            continue

        inputs = tokenizer(valid_prompts, padding=True, truncation=True, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=10)
        sentiment_labels = [tokenizer.decode(output, skip_special_tokens=True).strip().lower() for output in outputs]

        sentiment_idx = 0
        for prompt in prompts:
            if prompt is None:
                sentiments.append(0)
            else:
                label = sentiment_labels[sentiment_idx]
                if "positive" in label:
                    sentiments.append(1)
                elif "negative" in label:
                    sentiments.append(-1)
                else:
                    sentiments.append(0)
                sentiment_idx += 1

    return sentiments

# ==============================================================================
# 3. Data Loading, Preprocessing & Feature Engineering
# ==============================================================================
def load_and_preprocess_data():
    """
    Loads and cleans the Amazon review and world cities datasets,
    taking a 1000-row sample for faster processing.
    """
    try:
        reviews_data = pd.read_csv("Amazon_job_reviews_(USA_ India)2008-2020.csv")
        world_cities = pd.read_csv("worldcities.csv")
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure the CSV files are in the same directory.")
        return None, None

"""    # --- CRITICAL FIX: SAMPLE DATA HERE BEFORE ANY OTHER PROCESSING ---
    if len(reviews_data) > 1000:
        reviews_data = reviews_data.sample(n=1000, random_state=42)
        print(f"\nSampling complete: Using 1000 rows for the entire analysis pipeline.")"""
    # ------------------------------------------------------------------

    # Clean column names
    reviews_data.columns = reviews_data.columns.str.strip()

    print("--- Raw Data Insights (Sampled) ---")
    print("Shape of the reviews data:", reviews_data.shape)
    print("\nFirst 5 rows of the data:")
    print(reviews_data.head())

    # FIX 2: Correct Date Format ('%y') and drop critical NaNs
    reviews_data['Date'] = pd.to_datetime(reviews_data['Date'], format="%d %b %y", errors='coerce')
    reviews_data.dropna(subset=['Date', 'Location', 'Position', 'Overall rating'], inplace=True)
    reviews_data['Year'] = reviews_data['Date'].dt.year.astype(int)

    # Feature Engineering
    reviews_data['is_current_employee'] = reviews_data['Current employee'].apply(lambda x: 1 if x == 'Current employee' else 0)

    # Define columns for robust cleaning
    numerical_rating_cols = ['Work/Life Balance', 'Career Opportunities', 'Compensation and Benefits', 'Senior Management']
    zero_fill_cols = ['Culture and Values', 'Diversity and Inclusion']
    categorical_cols = ['CEO Approval', 'Recommended', 'Business Outlook']
    error_strings = ['#NAME?', '?', 'n/a', 'none', 'nil', ' ']

    # 3. ROBUST NUMERICAL IMPUTATION (Mean Filling)
    for col in numerical_rating_cols:
        reviews_data[col] = reviews_data[col].fillna(reviews_data[col].mean())

    # 4. ROBUST ZERO IMPUTATION
    for col in zero_fill_cols:
        reviews_data[col] = pd.to_numeric(reviews_data[col], errors='coerce')
        reviews_data[col] = reviews_data[col].fillna(0)

    # 5. ROBUST CATEGORICAL IMPUTATION
    for col in categorical_cols:
        reviews_data[col] = reviews_data[col].replace('nan', np.nan)
        reviews_data[col] = reviews_data[col].replace(error_strings, 'unknown')
        reviews_data[col] = reviews_data[col].fillna('unknown')

    # Drop Timeline column
    reviews_data.drop('Timeline', axis=1, inplace=True)

    # Handle text columns and fill with explicit placeholders
    reviews_data['pros'] = reviews_data['pros'].fillna("no text provided").str.lower()
    reviews_data['cons'] = reviews_data['cons'].fillna("no text provided").str.lower()

    reviews_data['Comment for company'] = reviews_data['Comment for company'].replace(error_strings, 'no comment provided')
    reviews_data['Comment for company'] = reviews_data['Comment for company'].fillna('no comment provided').str.lower()

    reviews_data['advice to Management'] = reviews_data['advice to Management'].replace(error_strings, 'no advice provided')
    reviews_data['advice to Management'] = reviews_data['advice to Management'].fillna('no advice provided').str.lower()

    # Encode categorical columns
    reviews_data['CEO_Approval_Score'] = reviews_data['CEO Approval'].map({'yes': 1, 'may be': 0.5, 'no': 0, 'unknown': 0.5})
    reviews_data['Recommended_Score'] = reviews_data['Recommended'].map({'yes': 1, 'may be': 0.5, 'no': 0, 'unknown': 0.5})
    reviews_data['Business_Outlook_Score'] = reviews_data['Business Outlook'].map({'yes': 1, 'may be': 0.5, 'no': 0, 'unknown': 0.5})

    reviews_data.info()

    return reviews_data, world_cities


def generate_wordcloud(text_series, title, filename):
    """Generates and saves a word cloud from a pandas Series of text."""
    text = " ".join(review for review in text_series if isinstance(review, str) and review.strip() not in ["na", "nan", "none", "#name?"])
    if not text:
        print(f"No valid text available for the {title} word cloud.")
        return
    stopwords = ["the", "and", "a", "to", "of", "is", "for", "in", "it", "with", "be", "not", "on", "that", "i", "you", "they", "we", "pros", "cons", "amazon", "company", "work", "job", "good", "great", "nice", "time", "people"]
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords).generate(text)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(title, fontsize=20)
    ax.axis('off')
    save_chart(fig, filename)


def perform_analysis(reviews_data, world_cities):
    """
    Performs all analysis steps and generates charts.
    Returns the processed reviews_data DataFrame and the correlation DataFrame.
    """
    print("Starting analysis and visualizations...")

    reviews_data.dropna(subset=['Year'], inplace=True)
    reviews_data['Year'] = reviews_data['Year'].astype(int)

    # All subsequent LLM calls run on the 1000-row sample
    print("\nPerforming Sentiment Analysis with LLM...")
    reviews_data['pros_sentiment'] = get_llama_sentiment(reviews_data['pros'].tolist())
    reviews_data['cons_sentiment'] = get_llama_sentiment(reviews_data['cons'].tolist())
    reviews_data['comment_sentiment'] = get_llama_sentiment(reviews_data['Comment for company'].tolist())
    reviews_data['advice_sentiment'] = get_llama_sentiment(reviews_data['advice to Management'].tolist())
    reviews_data['overall_satisfaction'] = reviews_data[['pros_sentiment', 'comment_sentiment']].mean(axis=1)

    print("\n--- After LLM Sentiment Analysis ---")
    print("New sentiment columns added:")
    print(reviews_data[['pros', 'pros_sentiment', 'cons_sentiment', 'comment_sentiment']].head())
    print("\n------------------------------------")

    print("\nGenerating Word Clouds...")
    generate_wordcloud(reviews_data['pros'], "Most Frequent Pros", "pros_wordcloud")
    generate_wordcloud(reviews_data['cons'], "Most Frequent Cons", "cons_wordcloud")

    print("\nGenerating Employee Location Map...")
    def get_coordinates():
        geolocator = Nominatim(user_agent="amazon_reviews_app")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        employee_count_by_city = reviews_data['Location'].value_counts().reset_index()
        employee_count_by_city.columns = ['City_Name', 'Employee_Count']
        matched_data = pd.merge(employee_count_by_city, world_cities[['city_ascii', 'lat', 'lng']],
                                left_on='City_Name', right_on='city_ascii', how='left')
        matched_data.dropna(subset=['lat', 'lng'], inplace=True)
        return matched_data
    map_data = get_coordinates()
    fig = px.scatter_geo(map_data, lat='lat', lon='lng', color="Employee_Count",
                         hover_name="City_Name", size="Employee_Count",
                         projection="natural earth", title="Employee Locations (USA and India)")
    save_chart(fig, "employee_map")

    print("\nGenerating Overall Satisfaction Histogram...")
    reviews_data['sentiment_category'] = reviews_data['overall_satisfaction'].apply(
        lambda x: 'Satisfied' if x > 0 else ('Dissatisfied' if x < 0 else 'Neutral')
    )
    sentiment_counts = reviews_data.groupby(['Year', 'sentiment_category']).size().unstack(fill_value=0)
    sentiment_counts = sentiment_counts.reindex(columns=['Satisfied', 'Dissatisfied', 'Neutral'], fill_value=0)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=sentiment_counts.index, y=sentiment_counts['Satisfied'], name='Satisfied', marker_color='lightgreen'))
    fig.add_trace(go.Bar(x=sentiment_counts.index, y=sentiment_counts['Dissatisfied'], name='Dissatisfied', marker_color='salmon'))
    fig.add_trace(go.Bar(x=sentiment_counts.index, y=sentiment_counts['Neutral'], name='Neutral', marker_color='lightgrey'))
    fig.update_layout(barmode='stack', title='Overall Employee Satisfaction by Year', xaxis_title='Year', yaxis_title='Number of Reviews')
    fig.update_xaxes(dtick=1)
    save_chart(fig, "satisfaction_histogram")

    print("\nGenerating Sentiment Analysis Over Time Chart...")
    sentiment_summary = reviews_data.groupby('Year').agg(
        avg_pros=('pros_sentiment', 'mean'),
        avg_cons=('cons_sentiment', 'mean'),
        avg_comment=('comment_sentiment', 'mean')
    ).reset_index()

    fig = go.Figure()
    fig.add_trace(go.Bar(x=sentiment_summary['Year'], y=sentiment_summary['avg_pros'], name='Pros Sentiment', marker_color='lightgreen'))
    fig.add_trace(go.Bar(x=sentiment_summary['Year'], y=sentiment_summary['avg_cons'], name='Cons Sentiment', marker_color='salmon'))
    fig.add_trace(go.Scatter(x=sentiment_summary['Year'], y=sentiment_summary['avg_comment'], mode='lines+markers', name='Comment for Company Trend', line=dict(color='grey', width=2)))
    fig.update_layout(title_text="Sentiment Analysis: Pros, Cons, and Comments Over Time", barmode='group')
    fig.update_xaxes(dtick=1)
    save_chart(fig, "sentiment_analysis")

    print("\nGenerating Ratings Impact Line Chart...")
    rating_columns = ['Work/Life Balance', 'Culture and Values', 'Diversity and Inclusion',
                      'Career Opportunities', 'Compensation and Benefits', 'Senior Management']
    yearly_correlations = []
    for year in reviews_data['Year'].unique():
        year_data = reviews_data[reviews_data['Year'] == year].dropna(subset=['Overall rating'] + rating_columns)
        if year_data.shape[0] > 1:
            for col in rating_columns:
                corr = year_data['Overall rating'].corr(year_data[col])
                yearly_correlations.append({'Year': year, 'Rating_Type': col, 'Correlation': corr})
    corr_df = pd.DataFrame(yearly_correlations)

    if corr_df.empty:
        print("Warning: Not enough valid data to generate 'Ratings Impact Line Chart'. Skipping.")
    else:
        fig = px.line(corr_df, x='Year', y='Correlation', color='Rating_Type', title='Impact of Ratings on Overall Rating Over Time')
        fig.update_xaxes(dtick=1)
        save_chart(fig, "ratings_impact_line_chart")

    print("\nGenerating Organizational Health Radar Chart...")
    avg_scores = reviews_data.groupby('Year')[['CEO_Approval_Score', 'Recommended_Score', 'Business_Outlook_Score']].mean().reset_index()
    fig = go.Figure()
    for year in avg_scores['Year'].unique():
        year_data = avg_scores[avg_scores['Year'] == year]
        fig.add_trace(go.Scatterpolar(
            r=year_data[['CEO_Approval_Score', 'Recommended_Score', 'Business_Outlook_Score']].values[0],
            theta=['CEO Approval', 'Recommended', 'Business Outlook'],
            fill='toself',
            name=str(year)
        ))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
                      title="Organizational Health by Year")
    fig.update_xaxes(dtick=1)
    save_chart(fig, "organizational_health_radar_chart")

    print("\nGenerating LLM-based Advice Summary...")
    yearly_advice = reviews_data.groupby('Year')['advice to Management'].apply(
        lambda x: " ".join(review for review in x if review != "no text provided")
    ).to_dict()

    for year, advice in yearly_advice.items():
        if len(advice) > 100:
            prompt = f"Summarize the following employee advice to management for the year {year}:\n\n{advice}"
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            outputs = model.generate(**inputs, max_length=150, min_length=40)
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\n--- Summary for {year} ---")
            print(summary)
            with open(f"charts/advice_summary_{year}.txt", "w") as f:
                f.write(summary)
        else:
            print(f"\nNot enough data to summarize for {year}.")

    return reviews_data, corr_df


# ==============================================================================
# 6. Predictive Modeling and Evaluation
# ==============================================================================
def run_predictive_model(reviews_data):
    """
    Performs the full predictive modeling pipeline to predict employee churn.

    This includes:
    - Feature selection
    - Data splitting (train/test)
    - Conditional handling of class imbalance (SMOTE)
    - Training a RandomForestClassifier
    - Evaluating the model's performance with key metrics and visualizations
    - Predicting the probability of an employee staying
    """
    print("\n--- Predictive Modeling for Employee Churn ---")

    features = ['Overall rating', 'Work/Life Balance', 'Culture and Values', 'Diversity and Inclusion',
                'Career Opportunities', 'Compensation and Benefits', 'Senior Management',
                'CEO_Approval_Score', 'Recommended_Score', 'Business_Outlook_Score',
                'pros_sentiment', 'cons_sentiment', 'comment_sentiment', 'advice_sentiment']
    target = 'is_current_employee'

    # Filter out any rows with NaN in the feature columns
    # The dataset size is already limited to ~1000 rows by load_and_preprocess_data
    model_data = reviews_data.dropna(subset=features + [target])

    X = model_data[features]
    y = model_data[target]

    # Split the data into training and test sets (70/30 split of the 1000-row sample)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    print(f"\nOriginal training set size: {X_train.shape}")
    print("Original class distribution:", Counter(y_train))

    # --- CRITICAL FIX: CONDITIONAL SMOTE APPLICATION ---
    if y_train.nunique() <= 1:
        print("--- WARNING: SMOTE Skipped ---")
        print("Training skipped SMOTE as only one class was found in the training sample.")
        X_train_resampled = X_train
        y_train_resampled = y_train
    else:
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
        print("Resampled training set size:", X_train_resampled.shape)
        print("Resampled class distribution:", Counter(y_train_resampled))

    # Initialize and train the Random Forest Classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train_resampled, y_train_resampled)

    # Make predictions on the original test set
    y_pred = rf_classifier.predict(X_test)

    # Predict the probability of an employee staying (class 1)
    y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

    # Evaluate the model
    print("\n--- Model Performance Metrics on Test Set ---")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Former Employee', 'Current Employee']))

    # Plot the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Former Employee', 'Current Employee'],
                yticklabels=['Former Employee', 'Current Employee'], ax=ax)
    ax.set_title("Confusion Matrix for Employee Churn Prediction")
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")
    save_chart(fig, "confusion_matrix")

    # Feature Importance
    feature_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns).sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=feature_importances, y=feature_importances.index, ax=ax, palette='viridis')
    ax.set_title("Feature Importance for Employee Churn Prediction")
    ax.set_xlabel("Importance")
    ax.set_ylabel("Feature")
    save_chart(fig, "feature_importance")

    print("\nPredictive modeling complete. Check the 'charts' directory for evaluation visuals.")
    print("\nExample predictions (first 5 test samples):")
    for i in range(5):
        print(f"Sample {i+1}: True Label={y_test.iloc[i]}, Predicted Label={y_pred[i]}, Predicted Probability of Staying={y_pred_proba[i]:.4f}")

# ==============================================================================
# 7. Main Execution Block
# ==============================================================================
if __name__ == "__main__":
    reviews_data, world_cities = load_and_preprocess_data()
    if reviews_data is not None and world_cities is not None:
        reviews_data_processed, corr_df = perform_analysis(reviews_data, world_cities)
        print("\n\n--- FINAL DATA PREVIEW BEFORE MODEL TRAINING ---")
        print("Final DataFrame with all features (Head):")
        print(reviews_data_processed.head())
        print("\nCorrelations DataFrame (Head):")
        print(corr_df.head())
        print("\n------------------------------------------------")
        run_predictive_model(reviews_data_processed)

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning: 


This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.

  from .kaleido import Kaleido


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


--- Raw Data Insights (Sampled) ---
Shape of the reviews data: (29494, 22)

First 5 rows of the data:
   ID number       Date       Location                   Position  \
0          1  22 Apr 08  Palo Alto, CA             Sales Director   
1          2  23 Apr 08  Lexington, KY      Systems Administrator   
2          3  02 May 08    Seattle, WA           Technical Writer   
3          4  23 May 08    Seattle, WA   Software Design Engineer   
4          5  25 May 08    Seattle, WA   Senior Marketing Manager   

                                 Comment for company  Overall rating  \
0  Amazon isn't all it's cracked up to be, unless...               2   
1                            Long hours and low pay.               2   
2       A fair website, but sucks as a place to work               2   
3  About what you'd expect when you sell your sou...               3   
4      I can't believe I get paid to do what I love!               4   

   Work/Life Balance  Culture and Values  Diversit

Getting Llama Sentiments (Batched):   0%|          | 0/905 [00:00<?, ?it/s]

Getting Llama Sentiments (Batched):   0%|          | 0/905 [00:00<?, ?it/s]